# EU press Scraping
**WIP file: There is a known issue of failure of scraping, fix is needed**

Source: [notebooks/module_4/week_9/scraping.ipynb](https://github.com/Data-Science-Public-Policy/graspp_2025_spring/blob/main/notebooks/module_4/week_9/scraping.ipynb)

In [1]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

## 1. Selenium
## 1.1 Scrape links

In [2]:
url = "https://ec.europa.eu/commission/presscorner/home/en?dotyp=&keywords=GHG&commissioner="
driver = webdriver.Chrome()
driver.get(url)

In [3]:
page_content = driver.find_element(By.ID, 'news-block')
# Now, find all <a> tags *within* that specific 'page_content' element
all_links= page_content.find_elements(By.TAG_NAME, 'a')

links = []
for link in all_links:
    href = link.get_attribute('href')
    text = link.text
    #print(f"Link Text: '{text}', URL: '{href}'")
    links.append(href)
links = [link for link in links if link!=None and len(link)>0]
links[:2]

['https://ec.europa.eu/commission/presscorner/detail/en/mex_25_1354',
 'https://ec.europa.eu/commission/presscorner/detail/en/ip_25_1337']

## 1.2 Scrape all links

In [6]:
#driver = webdriver.Chrome()
def scrape_page(url):
    driver.get(url)
    time.sleep(4)
    #page_title
    page_title = driver.find_element(By.TAG_NAME, 'h1').text
    #page_content
    page_content = driver.find_element(By.CLASS_NAME, 'ecl-col-l-9').text
    #date
    els = driver.find_elements(By.CLASS_NAME, 'ecl-page-header__meta-item')
    els = [el.text for el in els]
    date = next((item for item in els if re.search(r'\d{4}', item)), None)
    print(date)
    final_text = f"{page_title}\n{page_content}"
    return date.replace(",","").replace(" ", "_"), final_text

### 1.2.1 Example

In [7]:
date, final_text = scrape_page(links[1])
final_text[:400]

May 28, 2025


'\nShare this page:\nX\nFacebook\nLinkedin\nE-mail'

### 1.2.2 Scrape all

In [9]:
scrape_dictionary = {}
from tqdm import tqdm
for link in tqdm(links):
    try:
        date, final_text = scrape_page(link)
        scrape_dictionary[date] = final_text
    except:
        print("LINK DIDNT WORK", link)


  5%|▍         | 1/21 [00:15<05:02, 15.12s/it]

None
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/mex_25_1354


 10%|▉         | 2/21 [00:37<06:05, 19.26s/it]

May 28, 2025


 14%|█▍        | 3/21 [00:43<03:57, 13.19s/it]

...

None
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/ip_24_6523


 52%|█████▏    | 11/21 [02:20<02:31, 15.19s/it]

Dec 17, 2024


100%|██████████| 21/21 [02:21<00:00,  6.75s/it]

LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/ip_24_6464
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/ip_24_6346
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/ip_24_5967
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/ip_24_5605
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/ip_24_5467
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/mex_24_5470
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/qanda_24_5424
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/qanda_24_5382
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/qanda_24_2527
LINK DIDNT WORK https://ec.europa.eu/commission/presscorner/detail/en/ip_24_2322


### 1.2.3 Convert results to data frame

In [82]:
import pandas as pd
df_text = pd.DataFrame.from_dict(scrape_dictionary, orient = 'index', columns = ['text'])
df_text.index.name = 'date'
df_text.tail(2)

,text
date,
Apr_12_2024,Questions and Answers on the revised Energy Pe...
Apr_5_2024,Commission approves €350 million German State ...


### 1.2.4 Export

In [ ]:
import os
export_path = "../data/raw"
os.listdir(export_path)

[]

In [85]:
file_name = f"{export_path}/eu_press_releases_ghg.csv"
df_text.to_csv(file_name)

In [88]:
pd.read_csv(file_name).head(2)

,date,text
0,May_26_2025,Daily News 26 / 05 / 2025\nEU organises firefi...
1,May_20_2025,Commission approves €1.2 billion Dutch State a...
